# Updating the Inventory
---
This notebook will provide the code necessary to perform an update to the Biodata Resource Inventory, using trained models.

The steps include:
* Run new query on EuropePMC
* Classify new articles
* Run NER to get resource names for predicted positives
* Get URLs for predicted positives
* Gather other metadata



# Setup
---
### Mount Drive

First, mount Google Drive to have access to files necessary for the run:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/GitHub/inventory_2022/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/inventory_2022


Run the make target to install Python dependencies.

You may see the error: `ERROR: pip's dependency resolver does not currently take into account all the packages that are installed`, but the code should run regardless.



In [ ]:
! make setup_for_updating

If you need to download the model checkpoints for the best classifier and NER models, run the cell below. If the `train_and_predict` pipeline was run, then the models are already present.

In [ ]:
# Get trained article classification model
# Create output directory
! mkdir -p out/classif_train_out/best
# Print name of model to the necessary file
! CLASSIFIER="out/classif_train_out/article_classifier.pt"
! echo $CLASSIFIER > out/classif_train_out/best/best_checkpt.txt
# Download the model
! wget -O $CLASSIFIER https://huggingface.co/globalbiodata/inventory/resolve/main/article_classifier.pt
# Check that it downloaded properly
! echo "5718a7f70becacb46d46501734c83aab81c86feec563594f6a25c116aa31b521 $CLASSIFIER" | sha256sum -c

# Get trained NER model
! mkdir -p out/ner_train_out/best
! NER="out/ner_train_out/named_entity_recognition.pt"
! echo $NER > out/ner_train_out/best/best_checkpt.txt
! wget -O $NER https://huggingface.co/globalbiodata/inventory/resolve/main/named_entity_recognition.pt
! echo "dc0bc8b4929e33da52bc92e12720260b392421883889e0a36c809cb0b5c40f5d $NER" | sha256sum -c

# Setting up Configurations

Before running the automated pipelines, first update the configuration file `config/update_inventory.yml`. It can be accessed in Google Drive, though you may need to download it and edit it in a text editor such as Notepad, then reupload it.

* **Europe PMC query publication date range**: These are stored as variables `query_from_date` and `query_to_date` in that file. Note that the dates are inclusive. For example to get papers published in 2022, both of those varibles should be 2022.
* **Previous inventory file**: During strict deduplication and flagging for manual review, the results of the previous inventory are taken into account. Specify the location of the most recent inventory output file in the variable `previous_inventory`.

# Running the pipeline
---
Now, we are ready to run the pipeline

## Run it

The following cell will run the pipeline described above. It may take a while, but GPU will speed it up a lot.

In [ ]:
! make update_inventory

# Selective Manual Review

After running the initial pipeline, the inventory has been flagged for selective manual review.

The file to be reviewed is located at:

`out/new_query/for_manual_review/predictions.csv`

Review the flagged columns according to the instruction sheet ([doi: 10.5281/zenodo.7768363](https://doi.org/10.5281/zenodo.7768363)), then place the manually reviewed file in the following folder:

`out/new_query/manually_reviewed/`

The file must still be named `predictions.csv`

# Processing Manual Review

Next, further processing is performed on the manually reviewed inventory.

In [ ]:
! make process_manually_reviewed_update

snakemake \
-s snakemake/update_inventory.smk \
--configfile config/update_inventory.yml \
-c 1 \
--until process_countries
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                  count    min threads    max threads
-----------------  -------  -------------  -------------
check_urls               1              1              1
get_epmc_meta            1              1              1
process_countries        1              1              1
total                    3              1              1

Select jobs to execute...

[Mon Jan 30 20:13:29 2023]
rule check_urls:
    input: out/new_query/processed_manual_review/predictions.csv
    output: out/new_query/url_checking/predictions.csv
    jobid: 11
    reason: Missing output files: out/new_query/url_checking/predictions.csv
    resources: tmpdir=/tmp

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:1

## Final inventory

The final inventory, including names, URLS, and metadata is found in the file:
*    `out/new_query/processed_countries/predictions.csv`